## Model Monitoring Setup: Data Capture, Baseline Generation, Deployment Preparation for Model Quality Monitoring

In [15]:
# Imports & Setup
import boto3
import json
import numpy as np
import botocore
from botocore.exceptions import ClientError

import sagemaker
from sagemaker import get_execution_role, Session
from sagemaker.model_monitor import DataCaptureConfig, DefaultModelMonitor
from sagemaker.model import Model
from sagemaker.sklearn.model import SKLearnModel

In [16]:
# Model Location and Inference Code
model_artifact = 's3://sagemaker-us-east-1-531690656306/model/logistic_model.tar.gz'
entry_point = 'inference.py'

In [23]:
# Initialize Session
session = Session()
role = get_execution_role()

# Initialize SageMaker low-level boto3 client
sagemaker_client = boto3.client('sagemaker', region_name='us-east-1')

# Define key variables
region = 'us-east-1'
bucket = 'sagemaker-us-east-1-531690656306'  # Thai's S3 bucket
model_artifact = f's3://{bucket}/model/logistic_model.tar.gz'  # Thai's model path
entry_point = 'inference.py'

print("SageMaker session initialized.")

SageMaker session initialized.


In [24]:
# Create SKLearn Model Object
sklearn_model = SKLearnModel(
    model_data=model_artifact,
    role=role,
    entry_point=entry_point,
    framework_version='0.23-1',
    sagemaker_session=session
)

In [25]:
# Create Data Capture Config
data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=f's3://{bucket}/data-capture',
    capture_options=['Request', 'Response']
)

In [26]:
# Deploy Function With If-Else Check
def deploy_if_not_exists(model, endpoint_name, instance_type, data_capture_config=None):
    try:
        sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
        print(f"Endpoint '{endpoint_name}' already exists. Skipping deployment.")
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == 'ValidationException':
            print(f"Endpoint '{endpoint_name}' not found. Deploying new endpoint...")
            model.deploy(
                initial_instance_count=1,
                instance_type=instance_type,
                endpoint_name=endpoint_name,
                data_capture_config=data_capture_config
            )
            print("Deployment completed.")
        else:
            raise

In [27]:
# Call Deployment Function
endpoint_name = 'cardio-logistic-monitor-endpoint'
deploy_if_not_exists(
    model=sklearn_model,
    endpoint_name=endpoint_name,
    instance_type='ml.m5.xlarge',
    data_capture_config=data_capture_config
)

Endpoint 'cardio-logistic-monitor-endpoint' already exists. Skipping deployment.


In [28]:
# Generate Baseline for Model Monitor
monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600
)

In [29]:
# Define Baseline URIs
baseline_data_uri = 's3://sagemaker-us-east-1-531690656306/cardio_data/cardio_engineered.csv'
baseline_results_uri = 's3://sagemaker-us-east-1-531690656306/cardio_data/baseline-results'

In [30]:
# Generate Baseline
baseline_job = monitor.suggest_baseline(
    baseline_dataset=baseline_data_uri,
    dataset_format={'csv': {'header': True}},
    output_s3_uri=baseline_results_uri,
    wait=True
)


Job Name:  baseline-suggestion-job-2025-06-06-07-24-20-695
Inputs:  [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-531690656306/cardio_data/cardio_engineered.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-531690656306/cardio_data/baseline-results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
..............2025-06-06 07:26:32.724364: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-06-06 07:26:32.724391: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror

Full model monitoring pipeline has been prepared by first creating the SKLearn model object, which wraps the trained logistic regression model and inference script so SageMaker can use it. Then I enable data capture, which collects real-time input data and predictions as the model serves traffic, creating a record of live inference data. After that, I deploy the model as an endpoint to make it accessible for inference while capturing data. Once deployed, I generate a baseline using the fully cleaned and preprocessed training dataset; this allows SageMaker to compute statistics and constraints that define the model’s expected data distributions. These baseline statistics become the reference point that future data will be compared against to detect drift or anomalies. This full setup ensures that once monitoring jobs are configured, SageMaker can automatically evaluate incoming data for shifts that may impact model accuracy or stability.

* Calculates <b>statistics</b> (distribution, min, max, mean, std, percentiles, etc.)
* Generates <b>constraints</b> (rules/thresholds learned from your dataset, e.g., feature X must be within certain boundaries)

### Check Constraint Files Exist

In [36]:
# Initialize S3 client
s3 = boto3.client('s3')

# Your bucket and baseline folder
bucket = 'sagemaker-us-east-1-531690656306'
baseline_prefix = 'cardio_data/baseline-results/'

# List objects in the baseline folder
response = s3.list_objects_v2(Bucket=bucket, Prefix=baseline_prefix)

# Check and print files
if 'Contents' in response:
    print("Baseline statistics and constraints found:")
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print("No baseline files found.")

Baseline statistics and constraints found:
cardio_data/baseline-results/constraints.json
cardio_data/baseline-results/statistics.json




## For Reference Only
### <u>File Locations Summary for Model Monitoring Setup</u>

#### Trained Model Artifact (used for deployment & endpoint)
`s3://sagemaker-us-east-1-531690656306/model/logistic_model.tar.gz`

#### Training Dataset (used for baseline generation)
`s3://sagemaker-us-east-1-531690656306/cardio_data/cardio_train.csv`

#### Engineered Dataset (cleaned and engineered)
`s3://sagemaker-us-east-1-531690656306/cardio_data/cardio_engineered.csv`

#### Baseline Results Output Folder (generated by model monitor baseline job)
`s3://sagemaker-us-east-1-531690656306/cardio_data/baseline-results/`

#### Inside baseline-results:
- `statistics.json`  
  `s3://sagemaker-us-east-1-531690656306/cardio_data/baseline-results/statistics.json`

- `constraints.json`  
  `s3://sagemaker-us-east-1-531690656306/cardio_data/baseline-results/constraints.json`

#### Data Capture Location (request/response payloads captured from endpoint)
`s3://sagemaker-us-east-1-531690656306/data-capture/`